In [1]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

In [2]:
loader = DirectoryLoader(
    path="I:\\Sridhar\\Data\\Budget",
    glob="*.pdf",
    loader_cls=PyPDFLoader
)

In [3]:
documents = loader.load()

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [6]:
chunks = splitter.split_documents(documents)

In [7]:
print(f"Total chunks: {len(chunks)}")

Total chunks: 134


In [8]:
for chunk in chunks[1:5]:
    print(chunk)

page_content='CONTENTS 
 
PART – A 
 Page No. 
Introduction 1 
Budget Theme 1 
Agriculture as the 1st engine 3 
MSMEs as the 2nd engine 6 
Investment as the 3rd engine 8 
A. Investing in People 8 
B. Investing in the Economy 10 
C. Investing in Innovation 14 
Exports as the 4th engine 15 
Reforms as the Fuel 16 
Fiscal Policy 18 
 
 
PART – B 
Indirect taxes 20 
Direct Taxes  23 
 
Annexure to Part-A 29 
Annexure to Part-B 31' metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-02-01T03:54:26+05:30', 'author': 'hss', 'moddate': '2025-02-01T03:56:01+05:30', 'title': '', 'source': 'I:\\Sridhar\\Data\\Budget\\budget_speech.pdf', 'total_pages': 60, 'page': 2, 'page_label': '3'}
page_content='Budget 2025-2026 
 
Speech of 
Nirmala Sitharaman 
Minister of Finance 
February 1, 2025 
Hon’ble Speaker,  
 I present the Budget for 2025-26. 
Introduction 
1. This Budget continues our Government’s efforts to: 
a) accelerate growth,  
b) secure incl

In [ ]:
%pip install -qU langchain-google-vertexai chromadb --user

In [ ]:
!pip install blosc2

In [ ]:
!pip install --upgrade Cython==0.29.21

In [9]:
from langchain_google_vertexai import VertexAIEmbeddings

In [10]:
import os
os.environ["GOOGLE_CLOUD_PROJECT"] = "747074493049"

In [12]:
embedding_model = "text-embedding-005"
embeddings = VertexAIEmbeddings(model_name=embedding_model)


In [13]:
from langchain.vectorstores import Chroma

chroma_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)


In [14]:
retriever = chroma_store.as_retriever(search_kwargs={"k": 3})

In [15]:
results = retriever.invoke("What is the amount allocated for defence")

In [16]:
for result in results:
    print(result.page_content)

decriminalize more than 100 provisions in various laws.   
Fiscal Policy 
108. Now I move to fiscal policy matters.  
Fiscal Consolidation 
109. In the July Budget, I had committed to staying the course for fiscal 
consolidation. Our endeavour will be to keep the fiscal deficit each year such 
that the Central Government debt remains on a declining path as a percentage 
of the GDP. The roadmap for the  next 6 years has been detailed in the FRBM 
statement.           
Revised Estimates 2024-25 
110. The Revised Estimate of the total receipts other than borrowings is  
` 31.47 lakh crore, of which the net tax receipts are ` 25.57 lakh crore. The 
Revised Estimate of the total expenditure is ` 47.16 lakh crore, of which the 
capital expenditure is about ` 10.18 lakh crore. 
111. The Revised Estimate of the fiscal deficit is 4.8 per cent of GDP.
decriminalize more than 100 provisions in various laws.   
Fiscal Policy 
108. Now I move to fiscal policy matters.  
Fiscal Consolidation 
109. I

In [17]:
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.
